Data Source:

De Cock, D. (2011). Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project. Journal of Statistics Education, 19(3). https://doi.org/10.1080/10691898.2011.11889627

In [1]:
from pyreal.sample_applications import ames_housing

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In this tutorial, we use Pyreal to get feature contribution explanations for the AmesHousing dataset

First, we load in the data. Pyreal expects all data as DataFrames, where columns have the feature names.

In [2]:
x_orig, y = ames_housing.load_data(include_targets=True)
x_orig

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal
1,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,20,FV,62.0,7500,Pave,Pave,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,10,2009,WD,Normal
1456,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1457,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1458,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal


Next, we load in the interpretable feature descriptions.

In [3]:
feature_descriptions = ames_housing.load_feature_descriptions()
feature_descriptions

{'MSSubClass': ' Identifies the type of dwelling involved in the sale.',
 'MSZoning': ' Identifies the general zoning classification of the sale.',
 'LotFrontage': ' Linear feet of street connected to property',
 'LotArea': ' Lot size in square feet',
 'Street': ' Type of road access to property',
 'Alley': ' Type of alley access to property',
 'LotShape': ' General shape of property',
 'LandContour': ' Flatness of the property',
 'Utilities': ' Type of utilities available',
 'LotConfig': ' Lot configuration',
 'LandSlope': ' Slope of property',
 'Neighborhood': ' Physical locations within Ames city limits',
 'Condition1': ' Proximity to various conditions',
 'Condition2': ' Proximity to various conditions (if more than one is present)',
 'BldgType': ' Type of dwelling',
 'HouseStyle': ' Style of dwelling',
 'OverallQual': ' Rates the overall material and finish of the house',
 'OverallCond': ' Rates the overall condition of the house',
 'YearBuilt': ' Original construction date',
 'Ye

Next, we load the transfomers.

The first kind of transformer manually imputes the data based on information we know about the dataset.
We will call this the `AmesHousingImputer`

This imputation code comes from https://www.kaggle.com/juliencs/a-study-on-regression-applied-to-the-ames-dataset

In [4]:
transformers = ames_housing.load_transformers()
transformers

In [5]:
model = ames_housing.load_model()
model

Ridge()

Now, we can initialize a RealApp object.

In [6]:
from pyreal import RealApp

realApp =  RealApp(model,
                   x_orig,
                   y_train=y,
                   transformers=transformers,
                   feature_descriptions=feature_descriptions,
                   id_column="Id"
                )

We can make predictions using RealApp objects.

In [7]:
realApp.predict(x_orig.iloc[0:10])


X has feature names, but Ridge was fitted without feature names


{0: 157585.2499694071,
 1: 208451.08625945353,
 2: 200785.2076056447,
 3: 208661.747407954,
 4: 166794.2494034586,
 5: 285660.2014755854,
 6: 156554.26930558967,
 7: 279133.15592026466,
 8: 225810.47104341377,
 9: 138028.17155712727}

In [8]:
realApp.produce_feature_contributions(x_orig.iloc[0:10])


{0:                                          Feature Name Feature Value  \
 0    Identifies the type of dwelling involved in t...            20   
 1         Linear feet of street connected to property          75.0   
 2                             Lot size in square feet          9937   
 3    Rates the overall material and finish of the ...             5   
 4            Rates the overall condition of the house             6   
 ..                                                ...           ...   
 74                                       Pool quality           NaN   
 75                                      Fence quality           NaN   
 76   Miscellaneous feature not covered in other ca...           NaN   
 77                                       Type of sale            WD   
 78                                  Condition of sale        Normal   
 
     Contribution Average/Mode  
 0    1719.226755         47.0  
 1     219.032356    71.444444  
 2   -1529.914407      10374.8  

In [9]:
from pyreal.sample_applications import ames_housing
from pyreal.explainers.cf.counterfactual_examples import Counterfactuals
import time
import pandas as pd

start = time.time()
x_train_orig = ames_housing.load_data().drop(columns="Id")
model = ames_housing.load_model()
transformers = ames_housing.load_transformers()

input_row = x_train_orig.iloc[0:1]
explainer = Counterfactuals(model, x_train_orig, transformers=transformers, fit_on_init=True)
result = explainer.produce(input_row, target_prediction=200000)

cf = result.get_examples()
print(explainer.model_predict(cf))

Transformer class AmesHousingImputer does not have the required inverse explanation transform and is set to break, stopping transform process


[160089.64579568]


X has feature names, but Ridge was fitted without feature names


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,95,89.123615,9937,5,6,1892,1965,71.891654,825,290,...,CarPort,No,Gd,TA,Y,No,MnWw,Othr,ConLI,Alloca


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal


In [10]:
import numpy as np
def _dist(a, b):
    return np.linalg.norm(b - a)

x_model = np.array([0, 0, 0])
x = {"A":1, "B":0, "C":0}

x_df = pd.DataFrame(x, index=[0])[["A", "B", "C"]]
print(x_df)
x_np = x_df.to_numpy()
# Distance between x_mod and the input:
o2 = _dist(x_model, x_np)
# Sparsity of changes:
o3 = np.sum(x_model != x_np)
print(o2, o3)

   A  B  C
0  1  0  0
1.0 1
